In [15]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import HBox, VBox, interact, widgets
from IPython.display import display
from matplotlib import rcParams
import warnings

In [16]:
warnings.filterwarnings("ignore", category=UserWarning, message=".*missing from font.*")
rcParams['font.family'] = 'DejaVu Sans'
sns.set_theme() 

In [17]:
# Load data from SQLite database
def load_data():
    conn = sqlite3.connect('songs_data.db')

    # Updated query to join with the tables in the new schema
    query = '''
    SELECT 
        u.username,
        s.song_name,
        a.artist_name,
        h.liked,
        h.date,
        strftime('%Y', h.date) AS year,
        strftime('%m', h.date) AS month,
        strftime('%d', h.date) AS day,
        strftime('%H', h.date) AS hour,
        h.day_name
    FROM history h
    JOIN users u ON h.user_id = u.id
    JOIN songs s ON h.song_id = s.id
    JOIN artists a ON s.artist_id = a.id
    '''
    df = pd.read_sql_query(query, conn)
    conn.close()

    # Convert date to datetime for filtering
    df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S')

    # Convert 'year', 'month', 'day', and 'hour' columns to integers
    df['year'] = df['year'].astype(int)
    df['month'] = df['month'].astype(int)
    df['day'] = df['day'].astype(int)
    df['hour'] = df['hour'].astype(int)
    
    return df
    
df = load_data()

In [20]:
# Function to aggregate and visualize data
def visualize_data(aggregation='song', month=None, year=None, hour=None, user=None, liked=None, artist=None, day_number=None):
    # Load the data (make sure to call the load_data function)
    df = load_data()

    # Apply filters if specified
    if day_number is not None:
        # Ensure day_number is a tuple of two integers
        day_start, day_end = day_number
        if isinstance(day_start, int) and isinstance(day_end, int):
            df = df[(df['day'] >= day_start) & (df['day'] <= day_end)]
    
    if month:
        df = df[(df['month'] >= month[0]) & (df['month'] <= month[1])]
    if year:
        df = df[(df['year'] >= year[0]) & (df['year'] <= year[1])]
    if hour:
        df = df[(df['hour'] >= hour[0]) & (df['hour'] <= hour[1])]
    if user:
        df = df[df['username'] == user]
    if liked is not None:  # Check for True/False values
        df = df[df['liked'] == liked]
    if artist:
        df = df[df['artist_name'].str.contains(artist, na=False, case=False)]

    # Aggregation logic
    if aggregation == 'song':
        result = df.groupby('song_name').size().sort_values(ascending=False).head(10)
    elif aggregation == 'artist':
        result = df.groupby('artist_name').size().sort_values(ascending=False).head(10)
    elif aggregation == 'month':
        result = df.groupby('month').size().sort_values(ascending=False).head(10)
    elif aggregation == 'year':
        result = df.groupby('year').size().sort_values(ascending=False).head(10)
    elif aggregation == 'liked':
        result = df.groupby('liked').size().sort_values(ascending=False).head(10)
    elif aggregation == 'hour':
        result = df.groupby('hour').size().sort_values(ascending=False).head(10)
    elif aggregation == 'day_name':
        result = df.groupby('day_name').size().sort_values(ascending=False).head(10)
    else:
        result = pd.Series(dtype=int)  # Empty result for invalid aggregation

    # Plot the results
    if not result.empty:
        plt.figure(figsize=(10, 6))
        result.plot(kind='bar', color='skyblue')
        plt.title(f"Top {aggregation.capitalize()}s")
        plt.ylabel("Count")
        plt.xlabel(aggregation.capitalize())
        plt.xticks(rotation=45)
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.show()
    else:
        print("No data to display for the given filters.")

    # Display the aggregated data as output
    display(result)

   
# Interactive widget for visualization
def create_interactive_visualization():
    # Create the interactive widgets
    users = [None] + sorted(load_data()['username'].unique().tolist())

    aggregation = widgets.Dropdown(options=['song', 'artist', 'month', 'year', 'liked', 'day_name', 'hour'], 
                                   value='song', description='Aggregate by:')
    day_number = widgets.IntRangeSlider(value=[1, 31], min=1, max=31, step=1, description='Day range:')
    month = widgets.IntRangeSlider(value=[1, 12], min=1, max=12, step=1, description='Month range:')
    year = widgets.IntRangeSlider(value=[2020, 2024], min=2020, max=2024, step=1, description='Year range:')
    hour = widgets.IntRangeSlider(value=[0, 23], min=0, max=23, step=1, description='Hour range:')
    user = widgets.Dropdown(options=users, description='User:')
    liked = widgets.Dropdown(options=[None, True, False], description='Liked:')
    artist = widgets.Text(placeholder="Type artist name...", description="Artist:")

    # Arrange widgets horizontally
    widget_row1 = HBox([aggregation, day_number, month])
    widget_row2 = HBox([year, hour, user])
    widget_row3 = HBox([liked, artist])
    
    # Combine all rows vertically
    ui = VBox([widget_row1, widget_row2, widget_row3])
    
    # Link the UI with the function
    out = widgets.interactive_output(
        visualize_data, 
        {'aggregation': aggregation, 'day_number': day_number, 'month': month, 
         'year': year, 'hour': hour, 'user': user, 'liked': liked, 'artist': artist}
    )
    
    # Display the UI and output
    display(ui, out)

# Call the function to create the interactive visualization
create_interactive_visualization()


Output()